In [ ]:
from horton import *
import numpy as np
from numpy.linalg import norm
from numpy import dot,cross
import scipy as sp
Methane=IOData.from_file('Methane.xyz')
#print Methane.numbers
#Methane.coordinates*=angstrom
log.set_level(0)
CO=IOData.from_file('co.xyz')
NN=IOData.from_file('nn.xyz')

In [2]:
def uhf (mol,obasis,get_dms=False):
    lf=DenseLinalgFactory(obasis.nbasis)
    #orbital integrals
    olp=obasis.compute_overlap(lf)
    kin=obasis.compute_kinetic(lf)
    na=obasis.compute_nuclear_attraction(mol.coordinates,mol.pseudo_numbers,lf)
    er=obasis.compute_electron_repulsion(lf)
    
    exp_alpha=lf.create_expansion()
    exp_beta=lf.create_expansion()

    guess_core_hamiltonian(olp, kin, na, exp_alpha,exp_beta)
    Num_elec=sum(mol.numbers)
    occ_model=AufbauOccModel(int(Num_elec/2),int(Num_elec/2))
    occ_model.assign(exp_alpha, exp_beta)
    external = {'nn': compute_nucnuc(mol.coordinates, mol.pseudo_numbers)}
    terms = [
        UTwoIndexTerm(kin, 'kin'),
        UDirectTerm(er, 'hartree'),
        UExchangeTerm(er, 'x_hf'),
        UTwoIndexTerm(na, 'ne'),
    ]
    ham = UEffHam(terms, external)
    dm_alpha = exp_alpha.to_dm()
    dm_beta = exp_beta.to_dm()
    # - SCF solver
    scf_solver = EDIIS2SCFSolver(1e-10,maxiter=200)
    scf_solver(ham, lf, olp, occ_model, dm_alpha, dm_beta)
    
    grid=BeckeMolGrid(mol.coordinates,mol.numbers,mol.numbers, random_rotate=False,agspec='insane')
    rho_alpha = obasis.compute_grid_density_dm(dm_alpha, grid.points)
    rho_beta = obasis.compute_grid_density_dm(dm_beta, grid.points)
    rho=rho_alpha+rho_beta
    if get_dms:
        return ham.cache['energy'],rho,dm_alpha,dm_beta,ham.cache['dm_full'] 
    return ham.cache['energy'],rho 

In [7]:
def E_DM(mol,obasis,da,db,df):
    lf=DenseLinalgFactory(obasis.nbasis)
    #orbital integrals
    olp=obasis.compute_overlap(lf)
    kin=obasis.compute_kinetic(lf)
    na=obasis.compute_nuclear_attraction(mol.coordinates,mol.pseudo_numbers,lf)
    er=obasis.compute_electron_repulsion(lf)
    E0=compute_nucnuc(mol.coordinates, mol.pseudo_numbers)
    ### single electron terms
    E0+=df.contract_two('ab,ab',na)+df.contract_two('ab,ab',kin)
    #### two electron operatprs +++++++++  Coulomb and Hartree Fock exchange 
    E0+=er.contract_two_to_two('abcd,ac->bd',df).contract_two('ab,ab',df)*0.5
    E0-=er.contract_two_to_two('abcd,ab->cd',da).contract_two('ab,ab',da)*0.5
    E0-=er.contract_two_to_two('abcd,ab->cd',db).contract_two('ab,ab',db)*0.5
    return E0 

In [10]:
basis_set='def2-TZVP'

In [11]:
co_basis=get_gobasis(CO.coordinates,CO.numbers,basis_set)
nn_basis=get_gobasis(NN.coordinates,NN.numbers,basis_set)
en,r,da,db,df=uhf(CO,co_basis,get_dms=True)
print 'co energy',en
print 'nn with co_DM', E_DM(NN,nn_basis,da,db,df)
print 'nn true energy', uhf(NN,nn_basis)[0]
lf=DenseLinalgFactory(nn_basis.nbasis)
olp=nn_basis.compute_overlap(lf)
num_el=olp.contract_two('ab,ab',df) 
print 'number of electrons with new basis set',num_el
df._array*=14/olp.contract_two('ab,ab',df)
da._array*=7/olp.contract_two('ab,ab',da)
db._array*=7/olp.contract_two('ab,ab',db)
print 'nn with co_DM normalized', E_DM(NN,nn_basis,da,db,df)

co energy -112.7855220993662
nn with co_DM -108.55331528953053
nn true energy -108.97953010822461
number of electrons with new basis set 13.9744565447
nn with co_DM normalized -108.68104156924012


In [13]:
en,r,da,db,df=uhf(NN,nn_basis,get_dms=True)
print 'nn energy',en
print 'co with nn_DM', E_DM(CO,co_basis,da,db,df)
print 'co true energy', uhf(CO,co_basis)[0]
lf=DenseLinalgFactory(co_basis.nbasis)
olp=co_basis.compute_overlap(lf)
num_el=olp.contract_two('ab,ab',df) 
print 'number of electrons with new basis set',num_el
df._array*=14/olp.contract_two('ab,ab',df)
da._array*=7/olp.contract_two('ab,ab',da)
db._array*=7/olp.contract_two('ab,ab',db)
print 'co with nn_DM normalized', E_DM(CO,co_basis,da,db,df)

nn energy -108.97953010822461
co with nn_DM -112.33616840212746
co true energy -112.7855220993662
number of electrons with new basis set 13.9545719232
co with nn_DM normalized -112.57862608763449


In [4]:
basis_set='6-31G'
ch4_basis=get_gobasis(Methane.coordinates,Methane.numbers,basis_set)
N_basis=get_gobasis(Methane.coordinates[0:1],np.array([7]),basis_set)
alc_basis=GOBasis.concatenate(ch4_basis,N_basis)

 _ __ _
/ (..) \ Welcome to HORTON 2.1.1!
\/ || \/
 |_''_|  HORTON is written and maintained by by Toon Verstraelen (1).

         This version contains contributions from Toon Verstraelen (1), Pawel Tecmer (2),
         Farnaz Heidar-Zadeh (2), Cristina E. González-Espinoza (2), Matthew Chan (2),
         Taewon D. Kim (2), Katharina Boguslawski (2), Stijn Fias (3),
         Steven Vandenbrande (1), Diego Berrocal (2), and Paul W. Ayers (2)

         (1) Center for Molecular Modeling (CMM), Ghent University, Ghent, Belgium.
         (2) The Ayers Group, McMaster University, Hamilton, Ontario, Canada.
         (3) General Chemistry (ALGC), Free University of Brussels, Brussels, Belgium.

         More information about HORTON can be found on this website:
         http://theochem.github.com/horton/

         The purpose of this log file is to track the progress and quality of a
         computation. Useful numerical output may be written to a checkpoint
         file and is accessible 

In [6]:
uhf(Methane,alc_basis)

/home/giorgiod/.local/lib/python2.7/site-packages/horton/quadprog.py:621: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  lagrange = np.linalg.lstsq(r_free.T, -g_free)[0]


(-40.18152266777305,
 array([1.18246393e+02, 1.18246393e+02, 1.18246393e+02, ...,
        2.23452526e-59, 2.59747995e-67, 4.30643180e-51]))

In [7]:
gridc=BeckeMolGrid(Methane.coordinates,Methane.numbers,Methane.numbers, random_rotate=False,agspec='insane')

In [8]:
e0,rho0=uhf(Methane,alc_basis)

In [9]:
h=0.05
Methane.pseudo_numbers[0]+=h
Methane.pseudo_numbers[4]-=h
e1,rho1=uhf(Methane,alc_basis)

In [10]:
Methane.pseudo_numbers[0]-=2*h
Methane.pseudo_numbers[4]+=2*h
e2,rho2=uhf(Methane,alc_basis)

In [11]:
Methane.pseudo_numbers

array([5.95, 1.  , 1.  , 1.  , 1.05])

In [12]:
Ammonia=IOData()
Ammonia.coordinates=Methane.coordinates[:-1]
Ammonia.numbers=numbers=np.array([7,1,1,1])
Ammonia.pseudo_numbers=np.array([7.,1.,1.,1.])
nh3_basis=get_gobasis(Ammonia.coordinates,Ammonia.numbers,basis_set)
C_basis=get_gobasis(Methane.coordinates[0:1],np.array([6]),basis_set)
nh3c_basis=GOBasis.concatenate(nh3_basis,C_basis)

In [13]:
DVNN=compute_nucnuc(Ammonia.coordinates,Ammonia.pseudo_numbers)\
-compute_nucnuc(Methane.coordinates, Methane.pseudo_numbers)
DVNN

-2.435371246841294

In [14]:
e_nh3,rho_nh3 = uhf(Ammonia,nh3_basis)
e_nh3c,rho_nh3c = uhf(Ammonia,nh3c_basis)
e_nh3,e_nh3c

(-56.144426148514476, -56.15027240144965)

In [15]:
# alchemy second order in energy- first in density 
coordinates=Methane.coordinates
V1=((gridc.points[:, 0]-coordinates[0][0])**2+(gridc.points[:, 1]-coordinates[0][1])**2+\
    (gridc.points[:, 2]-coordinates[0][2])**2)**-0.5
V2=((gridc.points[:, 0]-coordinates[4][0])**2+(gridc.points[:, 1]-coordinates[4][1])**2+\
    (gridc.points[:, 2]-coordinates[4][2])**2)**-0.5
DV=V2-V1

In [16]:
E_est=e0+gridc.integrate(rho0,DV)+0.5*gridc.integrate(rho1-rho2,DV)/2./h
E_est,E_est+DVNN

(-53.934433310891116, -56.36980455773241)

In [20]:
E_est=e0+gridc.integrate(rho0,DV)+0.5*gridc.integrate(rho1-rho2,DV)/2/h\
+1/6.*gridc.integrate(rho1-2*rho0+rho2,DV)/h**2
e_nh3c,E_est+DVNN

(-56.15027240144965, -56.25127658764005)

.1 Hartree of error  -56.25561967818483 vs -56.15385511303025 